# Detecção e análise de rostos

Soluções de Pesquisa Visual Computacional geralmente exigem uma solução de inteligência artificial (IA) para poder detectar, analisar ou identificar rostos humanos. Por exemplo, suponha que a empresa de varejo Northwind Traders tenha decidido implementar uma "loja inteligente", em que serviços de IA monitoram o estabelecimento para identificar os clientes que precisam de ajuda e direcionar funcionários para auxiliá-los. Uma maneira de fazer isso é realizando detecção e análise facial (em outras palavras, determinar se existe algum rosto nas imagens e, em caso positivo, analisar suas características).

![Um robô analisando um rosto](./images/face_analysis.jpg)

## Usar o serviço cognitivo Detecção Facial para detectar rostos

Suponha que o sistema de loja inteligente que a Northwind Traders quer criar precise ser capaz de detectar clientes e analisar suas características faciais. No Microsoft Azure, você pode usar a **Detecção Facial**, parte dos Serviços Cognitivos do Azure para fazer isso.

### Criar um recurso dos Serviços Cognitivos

Vamos começar criando um recurso dos **Serviços Cognitivos** na sua assinatura do Azure.

> **Observação**: Se você já tiver um recurso dos Serviços Cognitivos, abra a página de **Início Rápido** no portal do Azure e copie a respectiva chave e o ponto de extremidade para a célula abaixo. Caso contrário, siga as etapas abaixo para criar um.

1. Em outra guia do navegador, abra o portal do Azure em https://portal.azure.com, entrando com sua conta Microsoft.
2. Clique no botão **&#65291;Criar um recurso**, procure *Serviços Cognitivos* e crie um recurso dos **Serviços Cognitivos** com as configurações abaixo:
    - **Assinatura**: *sua assinatura do Azure*.
    - **Grupo de recursos**: *Selecione ou crie um grupo de recursos com um nome exclusivo*.
    - **Região**: *Escolha qualquer região disponível*:
    - **Nome**: *Insira um nome exclusivo*.
    - **Tipo de preço**: S0
    - **Confirmo que li e entendi os avisos**: Selecionado.
3. Aguarde até que a implantação seja concluída. Depois, entre em seu recurso dos Serviços Cognitivos e, na página **Visão geral**, clique no link para gerenciar as chaves do serviço. Você precisará do ponto de extremidade e das chaves para se conectar aos seus recursos dos Serviços Cognitivos em aplicativos clientes.

### Obter a chave e o ponto de extremidade do seu recurso dos Serviços Cognitivos

Para usar seu recurso dos Serviços Cognitivos, os aplicativos clientes precisam do ponto de extremidade e da chave de autenticação:

1. No portal do Azure, na página **Chaves e ponto de extremidade** do seu recurso dos Serviços Cognitivos, copie a **Chave 1** do recurso e cole no código abaixo, substituindo **YOUR_COG_KEY**.

2. Copie o **ponto de extremidade** do recurso e cole no código abaixo, substituindo **YOUR_COG_ENDPOINT**.

3. Execute o código abaixo clicando no botão Executar célula <span>&#9655;</span> (no canto superior esquerdo da célula).

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Agora que você tem um recurso dos Serviços Cognitivos, pode usar o serviço de Detecção Facial para detectar rostos humanos na loja.

Execute a célula de código abaixo para ver um exemplo.

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces)

Cada rosto detectado recebe uma ID exclusiva, assim o aplicativo pode identificar cada um deles.

Execute a célula abaixo para ver as IDs dos rostos de outros compradores.

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

## Analisar atributos faciais

A Detecção Facial pode fazer muito mais do que simplesmente detectar rostos. Ela pode analisar características e expressões faciais para sugerir a idade e o estado emocional da pessoa. Por exemplo, execute o código abaixo para analisar os atributos faciais de um comprador.

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# Detect faces and specified facial attributes
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Display the faces and attributes (code in python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

Com base nas pontuações de emoção detectadas para o cliente na imagem, ele parece estar muito feliz com a experiência de compra.

## Encontrar rostos similares 

As IDs dos rostos que são criadas para face detectada são usadas para identificar detecções faciais individualmente. Você pode usar essas IDs para comparar um rosto detectado com rostos identificados anteriormente e com características semelhantes.

Por exemplo, execute a célula abaixo para comparar o comprador de uma imagem com compradores de outra e encontrar um rosto compatível.

In [ ]:
# Get the ID of the first face in image 1
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Get the face IDs in a second image
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Find faces in image 2 that are similar to the one in image 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Show the face in image 1, and similar faces in image 2(code in python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## Reconhecer rostos

Até agora, você viu que a Detecção Facial é capaz de detectar rostos e características faciais, além de identificar rostos parecidos. Você pode levar as coisas um pouco mais além implementando uma solução de *reconhecimento facial*, em que a Detecção Facial é treinada para reconhecer o rosto de uma pessoa específica. Isso pode ser útil em diversos cenários, como a marcação automática de fotos de amigos em um aplicativo de rede social ou o uso de reconhecimento facial como parte de um sistema de verificação biométrica de identidade.

Para ver como esse processo funciona, vamos supor que a empresa Northwind Traders queira usar o reconhecimento facial para garantir que somente funcionários autorizados do departamento de TI consigam acessar os sistemas de segurança.

Vamos começar criando um *grupo de pessoas* para representar os funcionários autorizados.

In [ ]:
group_id = 'employee_group_id'
try:
    # Delete group if it already exists
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

Agora que o *grupo de pessoas* existe, podemos adicionar uma *pessoa* para cada funcionário que quisermos incluir no grupo. Depois, registramos várias fotos de cada um deles para que a Detecção Facial aprenda quais são as características faciais distintas de cada pessoa. Idealmente, as imagens devem mostrar a mesma pessoa em poses diferentes e com expressões faciais diferentes.

Vamos adicionar um único funcionário chamado Wendell e registrar três fotos dele.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Add a person (Wendell) to the group
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Get photo's of Wendell
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Add each photo to person in person group
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Display each image
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

Com a pessoa adicionada e as fotografias registradas, podemos treinar a Detecção Facial para reconhecer cada indivíduo.

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

Agora, com o modelo treinado, você pode usá-lo para identificar rostos reconhecidos em uma imagem.

In [ ]:
# Get the face IDs in a second image
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Get recognized face names
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# show recognized faces
faces.show_recognized_faces(image_path, image_faces, face_names)



## Saiba mais

Para saber mais sobre o serviço cognitivo de Detecção Facial, consulte a [documentação da Detecção Facial](https://docs.microsoft.com/azure/cognitive-services/face/)
